In [31]:
import tensorflow as tf

import read
import prepare_data
import input_builder

import importlib
import model

BATCH_SIZE = 1
LEARNING_RATE = 2e-5
MAX_SEQ_LENGTH = 128

importlib.reload(input_builder)
import os
os.environ['TFHUB_CACHE_DIR'] = '/home/djjindal/bert/script-learning'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [32]:
dataset = 'dataset/gw_extractions_no_rep_no_fin.pickle'
train_dataset = read.read_data_iterator(dataset)
features = list(prepare_data.tokenize_if_small_enough(train_dataset,sentences=False,no_context=False))
train_set = features[:int(0.8 * len(features))]
val_set = features[int(0.8 * len(features)):int(0.9*len(features))]

In [34]:
print(train_set[0])

In [35]:
len(train_set)

7874

In [36]:
model_dir = 'output_triple'

In [37]:
run_config = tf.estimator.RunConfig(
    model_dir=model_dir,
    save_summary_steps=0,
    save_checkpoints_steps=0,
    log_step_count_steps=100)

In [38]:
model_fn = model.model_fn_builder(
  num_labels=5,
  learning_rate=LEARNING_RATE,
  num_train_steps=1,
  num_warmup_steps=1)

train_test_input_fn = input_builder.input_fn_builder(
    features=train_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=5)
val_test_input_fn = input_builder.input_fn_builder(
    features=val_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=5)
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'output_triple', '_tf_random_seed': None, '_save_summary_steps': 0, '_save_checkpoints_steps': 0, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f218cb7e9b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0421 15:54:39.952149 139784594450176 tf_logging.py:115] Using config: {'_model_dir': 'output_triple', '_tf_random_seed': None, '_save_summary_steps': 0, '_save_checkpoints_steps': 0, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f218cb7e9b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [39]:
# print(f'Evaluating Training Dataset!')
# print(estimator.evaluate(input_fn=train_test_input_fn))
# print(f'Evaluating Validation Dataset!')
# print(estimator.evaluate(input_fn=val_test_input_fn))

In [42]:
import numpy as np
def predict(sentenecs, triples, candidates, entity):
    e_dict = dict()
    check_dataset = []
    e_dict['sentences'] = sentenecs
    e_dict['triples'] = triples
    e_dict['candidates'] = candidates
    e_dict['correct'] = 4
    e_dict['entity'] = entity
    check_dataset = [e_dict]
    predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset, sentences=False))

    predict_input_fn = input_builder.input_fn_builder(
        features=predict_set,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False,
        candidates=5)
    predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False)

    for (i, prediction) in enumerate(predictions):
        arr = prediction['probabilities'][0]
        prob = [np.exp(arr[0]),np.exp(arr[1]),np.exp(arr[2]),np.exp(arr[3]),np.exp(arr[4])]
        total = np.sum(prob)
        max= np.max(prob)
        ec_dict = check_dataset[i]
        print("SENTENCE EVENT CHAIN", ec_dict['sentences'],"\n")
        print("TRIPLE EVENT CHAIN", ec_dict['triples'],"\n")
        print("CANDIDATES", ec_dict['candidates'],"\n")
        print("ENTITY", ec_dict['entity'],"\n")
        print("PREDICTION", prediction['labels'], "with Probability", np.max(prob),"\n","\n")
        return prediction['labels']

def on_button_clicked(b):
    print(text1.value, text2.value, text3.value)
    print(predict([], text1.value.split('), ('), text2.value.split('), ('), text3.value))    

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets
text1 = widgets.Text(description="Event Chain", width=200)
text2 = widgets.Text(description="Candidates", width=200)
text3 = widgets.Text(description="Entity", width=200)
button = widgets.Button(description="Predict")
button.on_click(on_button_clicked)
display(text1)
display(text2)
display(text3)
display(button)

# [('john','ordered',None),('john','ate','food')]
# [('john','left',None),('john','stays',None)]

# [('john','ordered',None),('john','ate','food'),('john','stood',None)]
# [('john','left',None),('john','stays',None)]

# [(None, 'took', 'phone call'), (None, 'had', 'something'), (None, 'began', 'to squint')] 
# [(None, 'broadcast', '(the two previous record-holders'), ('the pennant', 'slammed', None), (None, "n't imagine", 'Maris'), (None, 'expressed', 'interest'), (None, 'turned', '68 _')] 

Text(value='', description='Event Chain')

Text(value='', description='Candidates')

Text(value='', description='Entity')

Button(description='Predict', style=ButtonStyle())

[('john','ordered',None),('john','ate','food')] [('john','left',None),('john','stays',None)] John
INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Running local_init_op.


I0421 16:01:36.471080 139784594450176 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0421 16:01:36.613571 139784594450176 tf_logging.py:115] Done running local_init_op.


SENTENCE EVENT CHAIN [] 

TRIPLE EVENT CHAIN ["[('john','ordered',None),('john','ate','food'),('john','stood',None)]"] 

CANDIDATES ["[('john','left',None),('john','stays',None)]"] 

ENTITY John 

PREDICTION 1 with Probability 0.69223154 
 

1


In [2]:
# import pandas as pd

# common_sense_net = pd.read_hdf('mini.h5')
# common_sense_net = common_sense_net[common_sense_net.index.map(lambda x: x.startswith('/c/en/'))]
# common_sense_net.index = common_sense_net.index.map(lambda x: x.split('/c/en/')[1])

In [26]:
# common_sense_net.loc['UNK']

In [45]:
# dataset = list(read.read_data_iterator('dataset/gw_extractions_no_rep_no_fin.pickle'))
# train_data = dataset[:int(0.8 * len(features))]
# val_data = dataset[int(0.8 * len(features)):int(0.9*len(features))]

# check_dataset = []

# for i, ec_dict in zip(range(50), val_data):
#     check_dataset.append(ec_dict)
    
# predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset, sentences=False))
# predict_input_fn = input_builder.input_fn_builder(
#     features=predict_set,
#     seq_length=MAX_SEQ_LENGTH,
#     is_training=False,
#     drop_remainder=False,
#     candidates=5)

# predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False)
# print("Predict Set Length", len(predict_set))


In [8]:
# import numpy as np
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# for (i, prediction) in enumerate(predictions):
#     arr = prediction['probabilities'][0]
#     prob = [np.exp(arr[0]),np.exp(arr[1]),np.exp(arr[2]),np.exp(arr[3]),np.exp(arr[4])]
#     total = np.sum(prob)
#     max= np.max(prob)
# #     print("arg", prediction['labels'], "max ", np.max(prob))
#     ec_dict = check_dataset[i]
#     print("SENTENCE EVENT CHAIN", ec_dict['sentences'],"\n")
#     print("TRIPLE EVENT CHAIN", ec_dict['triples'],"\n")
#     print("CANDIDATES", ec_dict['candidates'],"\n")
#     print("ENTITY", ec_dict['entity'],"\n")
#     print("CORRECT", ec_dict['correct'] + 1,"\n")
#     print("PREDICTION", prediction['labels'], "with Probability", np.max(prob),"\n","\n")


In [44]:
# e_dict = dict()
# check_dataset = []
# e_dict['sentences'] = ['The other day, Ettinger unabashedly took\na phone call in front of a visitor.', 'The caller had something to\nsell.', 'Then he began to squint, seemingly pondering.'] 
# e_dict['triples'] = [(None, 'took', 'phone call'), (None, 'had', 'something'), (None, 'began', 'to squint')] 
# e_dict['candidates'] = [(None, 'broadcast', '(the two previous record-holders'), ('the pennant', 'slammed', None), (None, "n't imagine", 'Maris'), (None, 'expressed', 'interest'), (None, 'turned', '68 _')] 
# e_dict['correct'] = 4
# e_dict['entity'] = "Ettinger"
# check_dataset = [e_dict]
# predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset, sentences=False))

# predict_input_fn = input_builder.input_fn_builder(
#     features=predict_set,
#     seq_length=MAX_SEQ_LENGTH,
#     is_training=False,
#     drop_remainder=False,
#     candidates=5)
# predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False)

# for (i, prediction) in enumerate(predictions):
#     arr = prediction['probabilities'][0]
#     prob = [np.exp(arr[0]),np.exp(arr[1]),np.exp(arr[2]),np.exp(arr[3]),np.exp(arr[4])]
#     total = np.sum(prob)
#     max= np.max(prob)
#     ec_dict = check_dataset[i]
#     print("SENTENCE EVENT CHAIN", ec_dict['sentences'],"\n")
#     print("TRIPLE EVENT CHAIN", ec_dict['triples'],"\n")
#     print("CANDIDATES", ec_dict['candidates'],"\n")
#     print("ENTITY", ec_dict['entity'],"\n")
#     print("CORRECT", 4,"\n")
#     print("PREDICTION", prediction['labels'], "with Probability", np.max(prob),"\n","\n")
